<a href="https://colab.research.google.com/github/k-washi/python-cookbook/blob/master/ml/2/setReinforceLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
!pip uninstall tensorflow

In [0]:
!pip install tensorflow==2.0.0 tensorboard==2.0.1 tensorboardX==1.9

In [0]:
!pip install opencv-python==4.1.1.* pytorch-ignite==0.2.1 ptan==0.6 > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [6]:
tf.__version__

'2.0.0'

In [7]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [9]:
env = wrap_env(gym.make("MsPacman-v0"))
print(env.action_space)
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
show_video()

Discrete(9)


In [10]:
import gym
e = gym.make('CartPole-v0')
obs = e.reset()
print(obs) #x座標, 速度、地面に対する角度, 角速度
print(e.action_space) #右, 左
print(e.observation_space) 
print(e.step(0)) #(array([ 0.01626853, -0.19341435,  0.01088144,  0.29260441]), 1.0, False, {}) 新しい観測値, 報酬, done, 環境に関する追加状況

[-0.0067769   0.04100648 -0.04401741 -0.03822681]
Discrete(2)
Box(4,)
(array([-0.00595677, -0.15345752, -0.04478194,  0.24024972]), 1.0, False, {})


In [11]:
#ランダムなアクション、観測の取得
print(e.action_space.sample())
print(e.observation_space.sample())

print(e.action_space.sample())
print(e.observation_space.sample())

1
[-1.9842020e+00 -1.9095165e+38 -2.6278061e-01  2.3543422e+38]
0
[2.3568459e+00 1.5177587e+38 8.1440017e-02 2.4716929e+38]


上記の関数を流用した利用方法

In [0]:
env = gym.make("CartPole-v0")
total_reward = 0.0
total_steps = 0


In [13]:
obs = env.reset()
while True:
  action = env.action_space.sample()
  obs, reward, done, _ = env.step(action)
  total_reward += reward
  total_steps += 1
  if done:
    break

print(total_steps, total_reward)

14 14.0


# ラッパー

In [0]:
from typing import TypeVar
import random
Action = TypeVar('Action')

class RandomActionWrapper(gym.ActionWrapper):
  def __init__(self, env, epsilon=0.1):
    super(RandomActionWrapper, self).__init__(env)
    self.epsilon = epsilon
  def action(self, action:Action) -> Action:
    if random.random() < self.epsilon:
      print("Random!!")
      return self.env.action_space.sample()
    return action

In [0]:
env = RandomActionWrapper(gym.make("CartPole-v0"))

In [16]:
obs = env.reset()
total_reward = 0.0
while True:
  obs, reward, done, _ = env.step(0)
  total_reward += reward
  if done:
    break
print("Reward got: %.2f" % total_reward)

Reward got: 8.00


pytorch


In [18]:
import torch
import numpy as np

a = torch.FloatTensor(3,2)
a

tensor([[2.2262e-34, 0.0000e+00],
        [3.7835e-44, 0.0000e+00],
        [       nan, 0.0000e+00]])

In [19]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [20]:
#cuda
a = torch.FloatTensor([2,3])
ca = a.to('cuda')
ca

tensor([2., 3.], device='cuda:0')

# TensorboardX - NNの監視

In [0]:
import math
from tensorboardX import SummaryWriter

writer = SummaryWriter()
funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

for angle in range(-360, 360):
  angle_rad = angle * math.pi / 180
  for name, fun in funcs.items():
    val = fun(angle_rad)
    writer.add_scalar(name, val, angle)
writer.close()

runs配下に結果が格納される。

In [23]:
!tensorboard --logdir runs --bind_all

TensorBoard 2.0.1 at http://582693f1a089:6006/ (Press CTRL+C to quit)
ｑ
^C
